# 简介
本项目是参加飞桨常规赛：中文场景文字识别的项目。
生成的预测文件为work/PaddleOCR中的test2.txt文件

项目任务为识别包含中文文字的街景图片，准确识别图片中的文字

本项目源于https://aistudio.baidu.com/aistudio/projectdetail/615795，在此基础上进行修改

感谢开发者为开源社区做出的贡献

修改训练的模型的超参进行训练，通过实验进行验证。

# 赛题说明
**赛题背景**

中文场景文字识别技术在人们的日常生活中受到广泛关注，具有丰富的应用场景，如：拍照翻译、图像检索、场景理解等。然而，中文场景中的文字面临着包括光照变化、低分辨率、字体以及排布多样性、中文字符种类多等复杂情况。如何解决上述问题成为一项极具挑战性的任务。

本次飞桨常规赛以 中文场景文字识别 为主题，由2019第二届中国AI+创新创业全国大赛降低难度而来，提供大规模的中文场景文字识别数据，旨在为研究者提供学术交流平台，进一步推动中文场景文字识别算法与技术的突破。

**比赛任务**

要求选手必须使用飞桨对图像区域中的文字行进行预测，返回文字行的内容。

**数据集介绍**

本次竞赛数据集共包括33万张图片，其中21万张图片作为训练集，12万张作为测试集。数据集采自中国街景，并由街景图片中的文字行区域（例如店铺标牌、地标等等）截取出来而形成。所有图像都经过一些预处理，将文字区域利用仿射变化，等比映射为一张高为48像素的图片，如下图1所示：

![](https://ai-studio-static-online.cdn.bcebos.com/fb3cf59747e04f0cb9adde6a5a1945b3d9ef82f3b7c14c98bf248eb1c3886a3f)


(a) 标注：魅派集成吊顶

![](https://ai-studio-static-online.cdn.bcebos.com/57d58a35e1f34278bdb013b3f945ab69cddacf37c7fe40deba3c124fa1249753)


(b) 标注：母婴用品连锁
图1

**标注文件**

平台提供的标注文件为.txt文件格式。样例如下：



| h | w | name | value |
| -------- | -------- | -------- |-------- |
| 128 | 48 | img_1.jpg | 文本1|
| 56	| 48	| img_2.jpg|	文本2|
其中，文件中的四列分别是图片的宽、高、文件名和文字标注。

# 安装第三方库

将安装目录设置为external-libraries，这样项目重启后安装的库不会消失。

In [ ]:
!mkdir /home/aistudio/external-libraries
import sys
sys.path.append('/home/aistudio/external-libraries')
! pip install tqdm paddlepaddle-gpu==1.7.1.post97 -i https://mirror.baidu.com/pypi/simple
! pip install pqi
! pqi use aliyun
! pip install tqdm imgaug lmdb matplotlib opencv-python Pillow python-Levenshtein PyYAML trdg anyconfig # -t /home/aistudio/external-libraries

mkdir: cannot create directory ‘/home/aistudio/external-libraries’: File exists
Looking in indexes: https://mirror.baidu.com/pypi/simple
     |████████████████████████████████| 251.6MB 136kB/s  eta 0:00:01    |█████████                       | 70.4MB 8.9MB/s eta 0:00:21     |████████████████████████▉       | 195.3MB 9.0MB/s eta 0:00:07
  Found existing installation: paddlepaddle-gpu 1.6.2.post97
    Uninstalling paddlepaddle-gpu-1.6.2.post97:
      Successfully uninstalled paddlepaddle-gpu-1.6.2.post97
Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
  Created wheel for pqi: filename=pqi-2.0.6-cp37-none-any.whl size=4442 sha256=702ae86a214bc207575895558e6456f81000a1c6a7aa4029d9351b2fa86b1442
  Stored in directory: /home/aistudio/.cache/pip/wheels/54/76/37/bfae74b1d9f2b553cb45117a492ba3f567381e4f3ae307b5f6
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=dabfa5b82b3a141364fd95919f2aa2c2a1116c9fccb1818059977473c2b43f76
  Stored in direc

# 解压文件

压缩包内含训练集图片、训练集图片信息、测试集图片

In [ ]:
import os
os.chdir('/home/aistudio/data/data10879')
! tar -zxf train_img.tar.gz
! tar -zxf test_img.tar.gz 

In [ ]:
! unzip -oq /home/aistudio/data/data10879/test_images.zip -d /home/aistudio/data/data10879

unzip:  cannot find or open /home/aistudio/data/data10879/test_images.zip, /home/aistudio/data/data10879/test_images.zip.zip or /home/aistudio/data/data10879/test_images.zip.ZIP.


In [ ]:
train_opt = {
    "input_size": [1, 48, 256],
    "data_dir": "data/data10879",
    "train_dir": "train_images",
    "train_list": "train.txt",
    "label_list": "label_list.txt",
    "class_dim": -1,    # 样本数量 21W
    "label_dic": {},    
    "n_im": -1,    # 标签数量 3800+
    "continue_train": False,
    "save_model_dir": "work/crnn_model",
    "num_epochs": 120,
    "train_batch_size": 64,
    "mean_color": 127.0,
    "multi_data_reader_count": 4,
    "apply_distort": True,
    "image_distort_strategy": {
        "expand_prob": 0.3,
        "expand_max_ratio": 2.0,
        "hue_prob": 0.5,
        "hue_delta": 48,
        "contrast_prob": 0.5,
        "contrast_delta": 0.3,
        "saturation_prob": 0.5,
        "saturation_delta": 0.3,
        "brightness_prob": 0.5,
        "brightness_delta": 0.3,
    },
    "optimizer_strategy": {
        "learning_rate": 0.001,
        "momentum": 0.9,
        "lr_epochs": [40, 80],
        "lr_decay": [1, 0.5, 0.1],
    },
}

### 对数据进行扩充，增加训练数据。

In [ ]:
import numpy as np
import random
import cv2 as cv    # 导入opencv-python模块，opencv是一个强大的图像处理和计算机视觉库
from PIL import Image, ImageEnhance, ImageDraw    # python图像处理库
import matplotlib.pyplot as plt


def random_brightness(img):
    # 随机亮度可以改成固定亮度
    prob = np.random.uniform(0, 1)
    if prob < train_opt['image_distort_strategy']['brightness_prob']:
        brightness_delta = train_opt['image_distort_strategy']['brightness_delta']
        delta = np.random.uniform(-brightness_delta, brightness_delta) + 1
        img = ImageEnhance.Brightness(img).enhance(delta)
    return img


def random_contrast(img):
    # 随机对比度可以改成固定对比度
    prob = np.random.uniform(0, 1)
    if prob < train_opt['image_distort_strategy']['contrast_prob']:
        contrast_delta = train_opt['image_distort_strategy']['contrast_delta']
        delta = np.random.uniform(-contrast_delta, contrast_delta) + 1
        img = ImageEnhance.Contrast(img).enhance(delta)
    return img


def random_saturation(img):
    # 随机饱和度可以改成固定饱和度
    prob = np.random.uniform(0, 1)
    if prob < train_opt['image_distort_strategy']['saturation_prob']:
        saturation_delta = train_opt['image_distort_strategy']['saturation_delta']
        delta = np.random.uniform(-saturation_delta, saturation_delta) + 1
        img = ImageEnhance.Color(img).enhance(delta)
    return img


def random_hue(img):
    '''
    色调H  从红色开始，逆时针方向，绿色120°，蓝色240°；黄色60°，青色180°，品红300°    “红橙黄绿青蓝紫”
    饱和度S  0%--100%，饱和度越高颜色越鲜艳
    明度V  0%--100%  从黑到白
    '''
    # 随机色彩可以改成黑白  img = img.convert("L")
    prob = np.random.uniform(0, 1)
    if prob < train_opt['image_distort_strategy']['hue_prob']:
        hue_delta = train_opt['image_distort_strategy']['hue_delta']
        delta = np.random.uniform(-hue_delta, hue_delta)
        img_hsv = np.array(img.convert('HSV'))
        img_hsv[:, :, 0] = img_hsv[:, :, 0] + delta    # 以蓝色为基础，+85变成红色，+170变成绿色
        img = Image.fromarray(img_hsv, mode='HSV').convert('RGB')
    return img
    

def distort_image(img):
    prob = np.random.uniform(0, 1)
    # Apply different distort order
    # 可以删去，改成固定值
    if prob > 0.5:
        img = random_brightness(img) 
        img = random_contrast(img)
        img = random_saturation(img)
        img = random_hue(img)
    else:
        img = random_brightness(img)
        img = random_saturation(img)
        img = random_hue(img)
        img = random_contrast(img)
    return img


def rotate_image(img):
    """
    图像增强，增加随机旋转角度
    """
    prob = np.random.uniform(0, 1)
    if prob > 0.:
        angle = np.random.randint(-8, 8)
        img = img.convert('RGBA')
        img = img.rotate(angle, resample=Image.BILINEAR, expand=0)    # expand=True,保证旋转后图片的完整性
        fff = Image.new('RGBA', img.size, (127, 127, 127, 127))
        # 使用mask混合img1和img2
        # 若mask'mode为"L"，当L=255时全部显示img1，L=0时全部显示img2
        # 如果用灰色来混合原图，不如直接将原图转换成8位黑白格式
        img = Image.composite(img, fff, mask=img).convert('RGB')    # composite()要在"RGBA"下操作
    return img


def random_expand(img, keep_ratio=True):
    '''
    这一步可以删去
    '''
    if np.random.uniform(0, 1) < train_opt['image_distort_strategy']['expand_prob']:
        return img

    max_ratio = train_opt['image_distort_strategy']['expand_max_ratio']
    w, h = img.size
    c = 3
    ratio_x = random.uniform(1, max_ratio)
    # 改成x和y同时放大
    if keep_ratio:
        ratio_y = ratio_x
    else:
        ratio_y = random.uniform(1, max_ratio)
    oh = int(h * ratio_y)
    ow = int(w * ratio_x)
    # off_x, off_y  图像左上角在out_img的位置
    off_x = random.randint(0, ow - w)    
    off_y = random.randint(0, oh - h)

    out_img = np.zeros((oh, ow, c), np.uint8)
    for i in range(c):
        out_img[:, :, i] = train_opt['mean_color']
    
    # 这一步可能需要重新检测图像
    out_img[off_y: off_y + h, off_x: off_x + w, :] = img

    return Image.fromarray(out_img)


def preprocess(img):
    if train_opt['apply_distort']:
        img = distort_image(img)
    img_m = np.mean(img.convert('L'))
    img_std = max(np.std(img.convert('L')), 1e-2)
    # if train_opt['apply_distort']:
    #     img = random_expand(img)
    img = img.convert('L')

    return img

In [ ]:
import os
os.mkdir('/home/aistudio/data/data10879/train_images_tmp/')
filePath = '/home/aistudio/data/data10879/train_images_tmp'
for f in os.listdir(filePath):
    path = '/home/aistudio/data/data10879/train_images_tmp/'+f
    img = Image.open(path)
    if img.mode != 'RGB':    # 可以改成 'L'
        img = img.convert('RGB')
    # 图像增广并标准化
    img = preprocess(img)
    target = '/home/aistudio/data/data10879/train_images_tmp/'+f
    img.save(target)

# 预处理

* 文件 langconv(language convert)，这个文件用来把繁体字转成简体字<br>

* 函数 read_ims_list：读取train.list文件，生成图片的信息字典
* 函数 modify_ch：对标签label进行修改，进行四项操作，分别是“繁体->简体”、“大写->小写”、“删除空格”、“删除符号”。
* 函数 pipeline：调用定义的函数，对训练数据进行初步处理。

In [ ]:
from work.langconv import Converter
import codecs
import random
import sys
import os
from os.path import join as pjoin

os.chdir('/home/aistudio')
sys.path.append('/home/aistudio/work')
def read_ims_list(path_ims_list):
    """
    读取 train.list 文件
    """
    ims_info_dic = {}
    with open(path_ims_list, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split(maxsplit=3)
            w, h, file, label = parts[0], parts[1], parts[2], parts[3]
            ims_info_dic[file] = {'label': label, 'w': int(w)}
    return ims_info_dic
    

def modify_ch(label):
    # 繁体 -> 简体
    label = Converter("zh-hans").convert(label)

    # 大写 -> 小写
    label = label.lower()

    # 删除空格
    label = label.replace(' ', '')

    # 删除符号
    for ch in label:
        if (not '\u4e00' <= ch <= '\u9fff') and (not ch.isalnum()):
            label = label.replace(ch, '')

    return label

def save_txt(data, file_path):
    """
    将一个list的数组写入txt文件里
    :param data:
    :param file_path:
    :return:
    """
    if not isinstance(data, list):
        data = [data]
    with open(file_path, mode='w', encoding='utf8') as f:
        f.write('\n'.join(data))

def pipeline(dataset_dir):
    path_ims        = pjoin(dataset_dir, "train_images")
    path_ims_list   = pjoin(dataset_dir, "train.list")
    path_train_list = pjoin('/home/aistudio/work', "train.txt")
    path_test_list  = pjoin('/home/aistudio/work', "test.txt")
    path_label_list = pjoin('/home/aistudio/work', "dict.txt")

    # 读取数据信息
    file_info_dic = read_ims_list(path_ims_list)

    # 创建 train.txt
    class_set = set()
    data_list = []
    for file, info in file_info_dic.items():
        label = info['label']
        label = modify_ch(label)

        # 异常: 标签为空
        if label == '':
            continue

        for e in label:
            class_set.add(e)
        data_list.append("{0}\t{1}".format(pjoin('/home/aistudio/',path_ims, file), label))
        
    # 创建 label_list.txt
    class_list = list(class_set)
    class_list.sort()
    print("class num: {0}".format(len(class_list)))
    with codecs.open(path_label_list, "w", encoding='utf-8') as label_list:
        for id, c in enumerate(class_list):
            # label_list.write("{0}\t{1}\n".format(c, id))
            label_list.write("{0}\n".format(c))

    # 随机切分
    random.shuffle(data_list)
    val_len = int(len(data_list) * 0.05)
    val_list = data_list[-val_len:]
    train_list = data_list[:-val_len]
    print('训练集数量: {}, 验证集数量: {}'.format(len(train_list),len(val_list)))
    save_txt(train_list,path_train_list)
    save_txt(val_list,path_test_list)
    
random.seed(0)
pipeline(dataset_dir="data/data10879")

class num: 3827
训练集数量: 200342, 验证集数量: 10544


In [ ]:
os.chdir('/home/aistudio/work/PaddleOCR/')
!pwd

/home/aistudio/work/PaddleOCR


# 模型训练

In [23]:
! export PYTHONPATH=$PYTHONPATH:.
! python3 tools/train.py -c configs/rec/rec_r34_vd_none_bilstm_ctc.yml

2021-05-06 20:17:35,324-INFO: {'Global': {'algorithm': 'CRNN', 'use_gpu': True, 'epoch_num': 200, 'log_smooth_window': 20, 'print_batch_step': 100, 'save_model_dir': 'output/rec_CRNN_aug_341', 'save_epoch_step': 1, 'eval_batch_step': 1800, 'train_batch_size_per_card': 256, 'test_batch_size_per_card': 128, 'image_shape': [3, 32, 256], 'max_text_length': 64, 'character_type': 'ch', 'loss_type': 'ctc', 'reader_yml': './configs/rec/rec_icdar15_reader.yml', 'pretrain_weights': '/home/aistudio/work/PaddleOCR/model/latest', 'checkpoints': None, 'save_inference_dir': '/home/aistudio/work/test', 'character_dict_path': '/home/aistudio/work/dict.txt'}, 'Architecture': {'function': 'ppocr.modeling.architectures.rec_model,RecModel'}, 'Backbone': {'function': 'ppocr.modeling.backbones.rec_resnet_vd,ResNet', 'layers': 34}, 'Head': {'function': 'ppocr.modeling.heads.rec_ctc_head,CTCPredict', 'encoder_type': 'rnn', 'SeqRNN': {'hidden_size': 256}}, 'Loss': {'function': 'ppocr.modeling.losses.rec_ctc_los

# 模型预测

In [ ]:
!pwd

In [ ]:
! python3 tools/infer_rec.py -c configs/rec/rec_r34_vd_none_bilstm_ctc.yml -o Global.checkpoints=output/rec_CRNN_aug_341/best_accuracy

2021-05-06 20:07:34,311-INFO: {'Global': {'algorithm': 'CRNN', 'use_gpu': True, 'epoch_num': 200, 'log_smooth_window': 20, 'print_batch_step': 100, 'save_model_dir': 'output/rec_CRNN_aug_341', 'save_epoch_step': 1, 'eval_batch_step': 1800, 'train_batch_size_per_card': 256, 'test_batch_size_per_card': 128, 'image_shape': [3, 32, 256], 'max_text_length': 64, 'character_type': 'ch', 'loss_type': 'ctc', 'reader_yml': './configs/rec/rec_icdar15_reader.yml', 'pretrain_weights': '/home/aistudio/work/PaddleOCR/model/latest', 'checkpoints': 'output/rec_CRNN_aug_341/best_accuracy', 'save_inference_dir': '/home/aistudio/work/test', 'character_dict_path': '/home/aistudio/work/dict.txt'}, 'Architecture': {'function': 'ppocr.modeling.architectures.rec_model,RecModel'}, 'Backbone': {'function': 'ppocr.modeling.backbones.rec_resnet_vd,ResNet', 'layers': 34}, 'Head': {'function': 'ppocr.modeling.heads.rec_ctc_head,CTCPredict', 'encoder_type': 'rnn', 'SeqRNN': {'hidden_size': 256}}, 'Loss': {'function':